<a href="https://colab.research.google.com/github/shah-zeb-naveed/nlp/blob/main/nlp_pre_interview_refresher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP - Zero to Hero

In [9]:
# https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
# https://pberba.github.io/stats/2020/07/08/intro-hdbscan/

In [10]:
%pip install --upgrade pyLDAvis scikit-learn hdbscan scipy==1.10.1 svgling --quiet --progress-bar off

In [1]:
# prompt: import warning and disable

import warnings
warnings.filterwarnings('ignore')

In [2]:
import scipy
assert scipy.__version__ == '1.10.1'
from __future__ import print_function
import pyLDAvis
#import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
import re

#import torch
#from sentence_transformers import SentenceTransformer
#from bertopic import BERTopic
#from umap import UMAP

import hdbscan
import logging
logging.basicConfig()


/usr/local/lib/python3.11/dist-packages/hdbscan/plots.py:448: DeprecationWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/usr/local/lib/python3.11/dist-packages/hdbscan/robust_single_linkage_.py:154: DeprecationWarning: invalid escape sequence '\{'
  """Perform robust single linkage clustering from a vector array


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Define docs

In [4]:
from sklearn.datasets import fetch_20newsgroups
# docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data'][0:1000]

docs = [
    "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.",
    "The tower is 324 meters tall, about the same height as an 81-story building.",
    "It held this title for 41 years until the Chrysler Building in New York City was finished in 1930.",
    "The tower has three levels for visitors, with restaurants on the first and second levels.",
    "Tickets can be purchased to ascend by stairs or lift to the first and second levels.",
]


# # Save the documents to disk
# with open('docs.pkl', 'wb') as file:
#     pickle.dump(docs, file)

docs[0:1]

['The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.']

In [5]:
# with open('docs.pkl', 'rb') as file:
#     docs = pickle.load(file)

# Feature Extraction

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vectorizer = CountVectorizer()
vectorizer

CountVectorizer()

In [8]:
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
]
X = vectorizer.fit_transform(corpus)
X

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [9]:
analyze = vectorizer.build_analyzer()
analyze("This is a text document to analyze.")

['this', 'is', 'text', 'document', 'to', 'analyze']

In [10]:
vectorizer.get_feature_names_out()

X.toarray()

array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], dtype=object)

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]])

In [11]:
vectorizer.transform(['Something completely new.']).toarray() # ignored

array([[0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [12]:
# to preserve ordering to some extent can use bigrams
bigram_counts = CountVectorizer(ngram_range=(1,2))
bigram_counts.fit_transform(corpus).toarray()

array([[0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1]])

In [13]:
bigram_counts.get_feature_names_out()

array(['and', 'and the', 'document', 'first', 'first document', 'is',
       'is the', 'is this', 'one', 'second', 'second document',
       'second second', 'the', 'the first', 'the second', 'the third',
       'third', 'third one', 'this', 'this is', 'this the'], dtype=object)

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(smooth_idf=False)

In [15]:
counts = [[3, 0, 1],
          [2, 0, 0],
          [3, 0, 0],
          [4, 0, 0],
          [3, 2, 0],
          [3, 0, 2]]

tfidf = transformer.fit_transform(counts)
tfidf

tfidf.toarray() # each row a unit vector normalzied by eculidean norm

<6x3 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

array([[0.81940995, 0.        , 0.57320793],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.47330339, 0.88089948, 0.        ],
       [0.58149261, 0.        , 0.81355169]])

In [16]:
transformer = TfidfTransformer()
transformer.fit_transform(counts).toarray()

array([[0.85151335, 0.        , 0.52433293],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.55422893, 0.83236428, 0.        ],
       [0.63035731, 0.        , 0.77630514]])

In [17]:
transformer.idf_ # weifhts of each feature # maybe related to IDFs (some partial calculation)

array([1.        , 2.25276297, 1.84729786])

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(corpus).toarray()

array([[0.        , 0.43877674, 0.54197657, 0.43877674, 0.        ,
        0.        , 0.35872874, 0.        , 0.43877674],
       [0.        , 0.27230147, 0.        , 0.27230147, 0.        ,
        0.85322574, 0.22262429, 0.        , 0.27230147],
       [0.55280532, 0.        , 0.        , 0.        , 0.55280532,
        0.        , 0.28847675, 0.55280532, 0.        ],
       [0.        , 0.43877674, 0.54197657, 0.43877674, 0.        ,
        0.        , 0.35872874, 0.        , 0.43877674]])

In [19]:
# short texts noisty tfidf. binary occurrence as used in naive bayes can be set in CountVectorizer (binary=True)

In [20]:
# bow: cannot capture phrases/multi-words although can use ngrams, order, grammaer, potential misspellings, word derivations
# chars can help in misspellings

# analyzer is essetnial tokenizer

ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))
counts = ngram_vectorizer.fit_transform(['words', 'wprds'])
ngram_vectorizer.get_feature_names_out()
counts.toarray().astype(int)

array([' w', 'ds', 'or', 'pr', 'rd', 's ', 'wo', 'wp'], dtype=object)

array([[1, 1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0, 1]])

In [21]:
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(3, 3))
ngram_vectorizer.fit_transform(['jumpy fox'])
ngram_vectorizer.get_feature_names_out()


# across words. more noisy than white-space aware char_wb
ngram_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3))
ngram_vectorizer.fit_transform(['jumpy fox'])
ngram_vectorizer.get_feature_names_out()

<1x8 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

array([' fo', ' ju', 'fox', 'jum', 'mpy', 'ox ', 'py ', 'ump'],
      dtype=object)

<1x7 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

array([' fo', 'fox', 'jum', 'mpy', 'py ', 'ump', 'y f'], dtype=object)

In [22]:
ngram_vectorizer.vocabulary_

{'jum': 2, 'ump': 5, 'mpy': 3, 'py ': 4, 'y f': 6, ' fo': 0, 'fox': 1}

In [23]:
ngram_vectorizer.get_feature_names_out()

array([' fo', 'fox', 'jum', 'mpy', 'py ', 'ump', 'y f'], dtype=object)

In [24]:
ngram_vectorizer.get_stop_words()

## One-Hot Encoding

In [25]:
import torch

# Sample corpus
corpus = ["hello world", "hello", "world hello"]

# Step 1: Build the vocabulary
# Create a set of unique words
vocab = set()
for sentence in corpus:
    vocab.update(sentence.split())

# Create a word-to-index dictionary
word_to_index = {word: idx for idx, word in enumerate(vocab)}

# Step 2: Create one-hot representations
def one_hot_encode(word, word_to_index):
    """
    Create a one-hot vector for the given word.
    """
    vector = torch.zeros(len(word_to_index), dtype=torch.float32)
    vector[word_to_index[word]] = 1.0
    return vector

# Encode the entire corpus
one_hot_encoded_corpus = []
for sentence in corpus:
    encoded_sentence = []
    for word in sentence.split():
        encoded_sentence.append(one_hot_encode(word, word_to_index))
    one_hot_encoded_corpus.append(encoded_sentence)

one_hot_encoded_corpus

[[tensor([0., 1.]), tensor([1., 0.])],
 [tensor([0., 1.])],
 [tensor([1., 0.]), tensor([0., 1.])]]

In [26]:
ord('A')

65

In [27]:
#  This technique is useful for high-cardinality categorical features (e.g., URLs, words, or user IDs) where one-hot encoding is too memory-intensive.

from sklearn.feature_extraction import FeatureHasher
import pandas as pd

# Sample email subjects
emails = [
    {"free": 1, "money": 1, "win": 1},     # spam
    {"meeting": 1, "schedule": 1},         # ham
    {"lottery": 1, "winner": 1},           # spam
    {"project": 1, "update": 1}            # ham
]

# Use FeatureHasher (e.g., 8 hashed features)
hasher = FeatureHasher(n_features=8, input_type='dict')
X_hashed = hasher.transform(emails)

# Convert to array or DataFrame for viewing
df_hashed = pd.DataFrame(X_hashed.toarray())
print(df_hashed)

     0    1    2    3    4    5    6    7
0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  1.0  0.0 -1.0  0.0  0.0
2  0.0  0.0 -1.0  0.0  0.0  0.0 -1.0  0.0
3 -1.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0


In [28]:
# can implement custom hasher or use python's built-in
hash('test')

-3910120702538823191

# Embeddings

Word2vec (Google) - 2 techniques: Continuous Bag of Words (CBoW) and Skip-Gram;
Bow: use context of window size m to predict target. softmax layer and negative log-likelihood given context to train. faster.

Skip-gram: use center to predict context in window size m to predict context words. one output layer + softmax predicts (center, context) pair. Avg. neg. log likelihood to train. better for infrequent words.

Goal is to learn weights that are the vectors. Both Can use negative sampling to optimize (better for frequent). Hierarchical softmax (better for infrequent words).
Linear substructures like vector(man) - vector(woman) + vector(king) = vector(queen). Can average embeddings of multiple words to find closest vectors during inference.


Global Vectors or GloVe (Stanford); trained on co-occurence/frequency matrix of word pairs. Can use KNN to get similar words. Linear substructurs: vector difference of man-woman is similar to king-queen. This is encoded as its trained so its dot product equals log of prob ratio. This encodes meaning of the word e.g. ice/steam/gas/water example.

fastText (Facebook) —interesting fact: accounts for out of vocabulary words.


In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Example sentences
sentences = [
    "the quick brown fox jumps over the lazy dog",
    "I love natural language processing",
    "word2vec is a cool technique"
]

# Tokenize sentences
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Train Word2Vec model using CBOW
model_cbow = Word2Vec(tokenized_sentences, vector_size=100, window=2, min_count=1, sg=0) # switch skip-gram

# Get the word vector for a word
vector = model_cbow.wv['fox']
print(vector)

# Find most similar words
similar_words = model_cbow.wv.most_similar('fox')
print(similar_words)


In [ ]:
# # use pre-trained

# import gensim.downloader as api

# # Load the pre-trained Word2Vec model (Google's pretrained model)
# model = api.load("word2vec-google-news-300")
# model.most_similar('fox')

## Doc2Vec

- based on word2vec, shallow network. random init doc vvector to predict a sample of words in that document. results in static embeddings, treats all context words equally regardless of order/sposition. BERT is bidirection (processes words right and left), is deep, finetuend on sentence similarity, higher scores in benchmakrs,

In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Example documents
documents = [
    "I love machine learning.",
    "Gensim is a useful library for NLP.",
    "Doc2Vec is an extension of Word2Vec.",
    "This is an example document."
]

# Preprocess and tag the documents
tagged_documents = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(documents)]

# Initialize and train the Doc2Vec model
model = Doc2Vec(vector_size=50, window=2, min_count=1, workers=4, epochs=100)

# Build the vocabulary
model.build_vocab(tagged_documents)

# Train the model
model.train(tagged_documents, total_examples=model.corpus_count, epochs=model.epochs)

# Inference: Get the vector for a new document
new_doc = "Machine learning is fascinating."
new_vec = model.infer_vector(new_doc.split())

# Print the vector
print(f"Vector for the new document: {new_vec}")


## Spacy

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
# brief chatgpt example:

import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the SpaCy English model
nlp = spacy.load("en_core_web_md")

# Example set of documents
documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
    "This is not a document.",
    "We have more documents to explore."
]

# Function to convert documents to vectors
def document_vectors(documents):
    doc_vectors = []
    for doc in documents:
        doc_vector = nlp(doc).vector
        doc_vectors.append(doc_vector)
    return np.array(doc_vectors)

# Convert documents to vectors
doc_vectors = document_vectors(documents)

# Test document
test_doc = "More documents should be explored."

# Convert test document to vector
test_doc_vector = nlp(test_doc).vector

# Calculate cosine similarity between test document and each document in the set
similarities = cosine_similarity([test_doc_vector], doc_vectors)[0]

# Retrieve indices of most similar documents
most_similar_indices = similarities.argsort()[:-4:-1]

# Print most similar documents
print("Most similar documents to the test document:")
for idx in most_similar_indices:
    print(f"Document {idx}: {documents[idx]}")


In [ ]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.tag_, token.has_vector, token.vector_norm)

for ent in doc.ents:
  print(ent.text, ent.label_)

In [ ]:
doc1 = nlp("I like salty fries and hamburgers.")
doc2 = nlp("Fast food tastes very good.")

# Similarity of two documents
print(doc1, "<->", doc2, doc1.similarity(doc2))
# Similarity of tokens and spans
french_fries = doc1[2:4]
burgers = doc1[5]
print(french_fries, "<->", burgers, french_fries.similarity(burgers))


# Misc

In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """Calculate the cosine similarity between two vectors."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

# Example vectors
vec1 = np.array([1, 2, 3])
vec2 = np.array([4, 5, 6])

# Calculate cosine similarity
similarity = cosine_similarity(vec1, vec2)
print(f"Cosine Similarity: {similarity}")


In [ ]:
from collections import Counter
Counter('A quick brown fox jumps')

Counter(['Abc bcg', 'def'])

# NLTK Stuff

In [ ]:
import string
puncs = set(string.punctuation)
print(puncs)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
stop_words = set(stopwords.words('english')) # can also just specify in sklearn
len(stop_words)

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess(doc):
  tokens = [lemmatizer.lemmatize(word.lower()) for word in doc.split(' ') if word not in stop_words and word not in puncs]
  return tokens

tokenized_docs = [preprocess(doc) for doc in docs]
print(tokenized_docs)

In [ ]:
# since already tokenized, if wanna create BoW, can use:

vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False, smooth_idf=True, stop_words="english")
tfidf_matrix = vectorizer.fit_transform(tokenized_docs)

tfidf_matrix.toarray().shape
print(tfidf_matrix.toarray())

In [ ]:
nltk.download('punkt')

# for better handling of ambiguous situations like U.S.A.

from nltk.tokenize import sent_tokenize, word_tokenize

text = "Natural language processing is an exciting area. Huge budget have been allocated for this."
print(sent_tokenize(text))
print(word_tokenize(text))

In [ ]:
# pos tagging

nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag, word_tokenize, RegexpParser

# Find all parts of speech in above sentence
tagged = pos_tag(word_tokenize(text))
tagged

## Parse Trees

In [ ]:
# empty grammar example

# Extract all parts of speech from any text
chunker = RegexpParser("""
                    NP: {}   # To extract Noun Phrases
                    P: {}    # To extract Prepositions
                    V: {}    # To extract Verbs
                    PP: {}   # To extract Prepositional Phrases
                    VP: {}   # To extract Verb Phrases
                    """)

# Print all parts of speech in above sentence
output = chunker.parse(tagged)
print("After Extracting\n", output)


# output.draw() display error
display(output)

# another set of grammar rules:

#Extract all parts of speech from any text
chunker = RegexpParser("""
                    NP: {<DT>?<JJ>*<NN>} #To extract Noun Phrases
                    P: {<IN>}            #To extract Prepositions
                    V: {<V.*>}           #To extract Verbs
                    PP: {<p> <NP>}       #To extract Prepositional Phrases
                    VP: {<V> <NP|PP>*}   #To extract Verb Phrases
                    """)

# Print all parts of speech in above sentence
output = chunker.parse(tagged)
print("After Extracting\n", output)

# output.draw() display error
display(output)

In [ ]:
import nltk
from nltk import CFG

# CFGs can not be used with english as its not as expressive.

# Define a context-free grammar
grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Det N | 'I'
    VP -> V NP | V
    Det -> 'the' | 'a'
    N -> 'dog' | 'cat'
    V -> 'chased' | 'ate'
""")

# Create a recursive descent parser
parser = nltk.RecursiveDescentParser(grammar)

# Define a sentence to parse
sentence = "I chased the cat"

# Parse the sentence and print the parse trees
for tree in parser.parse(sentence.split()):
    print(tree)
    display(tree)


1. https://www.tutorialspoint.com/natural_language_processing/natural_language_processing_syntactic_analysis.htm
2. https://www.analyticsvidhya.com/blog/2022/03/syntactical-parsing-in-nlp/
3. https://intellipaat.com/blog/what-is-parsing-in-nlp/

# PyTorch

In [ ]:
import torch
import numpy as np

In [ ]:
t = torch.tensor([[1, 2, 3], [5, 2, 5]])
t

In [ ]:
t.t()

In [ ]:
t.shape

un_t = torch.unsqueeze(t, dim=0) # equivalent of np.expand(x, axis=0)
un_t

In [ ]:
un_t.squeeze()

In [ ]:
torch.sum(un_t).item()

In [ ]:
un_t.view(-1, 1, 1) # only one dimension can be inferred

In [ ]:
torch.zeros(2, 3, 5)

In [ ]:
x_ones = torch.ones_like(t) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(t, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

In [ ]:
t = torch.tensor([[1, 1], [1, 1]])

# This computes the matrix multiplication between two tensors. y1, y2 will have the same value
y1 = t @ t.T
y2 = t.matmul(t.T)
y1
y2

# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = t * t
z2 = t.mul(t)
z1
z2

In [ ]:
t.view(1,-1)

In [ ]:
t.view(1,-1).squeeze()
n = t.view(1,-1).squeeze().numpy()
np.dot(n, n)

In [ ]:
t
torch.cat((t, t, t), dim=1)

In [ ]:
t
t[-1, 0:2]

In [ ]:
(torch.tensor([[1, 2, 0, -1]]) == 1) # masking

In [ ]:
sz=3
mask = (torch.triu(torch.ones((sz, sz))) == 1).transpose(0, 1)
mask

In [ ]:
mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, 10)

In [ ]:
x = [1, 2, 3]
np.where(x == 1)[0]

In [ ]:
torch.sigmoid(torch.tensor(5))

torch.nn.ReLU()(torch.tensor(-4))

In [ ]:
X = torch.rand(2, 5)
X

In [ ]:
probs = torch.nn.Softmax(dim=-1)(torch.tensor([[9], [10], [5]], dtype=float))
probs

probs.argmax(0)

In [ ]:
# an ordered container
import torch.nn as nn

img_size = 28

# no need to define forward()
seq_modules = nn.Sequential(
    torch.nn.Flatten(),
    nn.ReLU(),
    nn.Linear(img_size * img_size, 10)
)

# 5 is probably channels I think
input_image = torch.rand(5,img_size,img_size)
logits = seq_modules(input_image)
logits.shape

In [ ]:
linear_layer = nn.Linear(4, 1)
for x in linear_layer.named_parameters():
  print('---------')
  print(x[0], x[1].numel())

In [ ]:
seq_modules = nn.Sequential(
    nn.Linear(3, 2),
    nn.Linear(2, 1)
)
for x in seq_modules.named_parameters():
  print('---------')
  print(x[0], x[1].numel())

In [ ]:
# can use within collate_fn in dataloader
import torch
from torch.nn.utils.rnn import pad_sequence

# Example list of sequences with varying lengths
sequences = [torch.tensor([1, 2, 3], dtype=torch.float),
             torch.tensor([4, 5], dtype=torch.float),
             torch.tensor([6], dtype=torch.float)]

# Pad the sequences
padded_sequence = pad_sequence(sequences, batch_first=True, padding_value=0.0)
padded_sequence

In [ ]:
# serves different purpose.
# ignores pad values and leads to better memory utilization

import torch
from torch.nn.utils.rnn import pack_padded_sequence

lengths = torch.tensor([len(t) for t in padded_sequence])

# Pack the padded sequence
packed_sequence = pack_padded_sequence(padded_sequence, lengths, batch_first=True)
packed_sequence

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset

# can't use tensordataset directly for variable input lengths
# need to implement class

# can't init tensor on variable. otherwise, just init as tensor and use tensordataset
X = [[1, 2, 3], [5, 7], [5]]

# needed
X = list(map(torch.tensor, X))

y = torch.tensor([0, 1, 1])

# can't use for variable
#dataset = TensorDataset(X, y)

class SequenceDataset(Dataset): # just a protocol class so doesn't inherit anythinmg no super()
  def __init__(self, sequences, labels):
    self.sequences = sequences
    self.labels = labels

  def __len__(self):
    return len(self.sequences)

  def __getitem__(self, idx):
    return self.sequences[idx], self.labels[idx]

dataset = SequenceDataset(X, y)

def collate_fn(batch):
  sequences, labels = zip(*batch)
  lengths = [len(seq) for seq in sequences]
  padded = pad_sequence(sequences, batch_first=True, padding_value=0)
  return padded, lengths, labels

dataloader = DataLoader(dataset, batch_size=2, collate_fn=collate_fn)

# Iterate through DataLoader
for batch in dataloader:
    padded_sequences, lengths, labels = batch
    print("Padded Sequences:\n", padded_sequences)
    print("Lengths:\n", lengths)
    print("Labels:\n", labels)

## Streaming Data

Can also use pandas(chunk_size) or maybe using numpy.

---



In [ ]:
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

file = 'example6.h5'

# Create an HDF5 file
with h5py.File(file, 'w') as f:
    f.create_dataset('data', data=np.arange(100), compression="gzip")
    f.create_dataset('labels', data=np.random.randint(0, 3, size=(100,)), compression="gzip")

class HDF5Dataset(Dataset):
    def __init__(self, h5_file):
        self.file = h5_file
        self.dataset = h5py.File(h5_file, 'r')
        self.data = self.dataset['data']
        self.labels = self.dataset['labels']
        #self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx], dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

# Usage example
h5_file = file
dataset = HDF5Dataset(h5_file)

In [ ]:
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

# Iterate over DataLoader in training loop
for inputs, labels in data_loader:
    # Perform training step
    print(inputs)
    print('===========')

## CBOW in PyTroch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import numpy as np

# Sample data
corpus = [
    "he is the king",
    "the king is royal",
    "she is the queen",
    "the queen is royal",
    "royal is nice",
    "she is nice"
]

# Tokenize corpus
words = set()
for sentence in corpus:
    for word in sentence.split():
      words.add(word)

#print([word for sent in corpus for word in sent.split(' ')])

In [ ]:
#word_counts = Counter(words)
vocab = {word: idx for idx, word in enumerate(words)}
idx_to_word = {idx: word for word, idx in vocab.items()}

In [ ]:
# CBOW Dataset
class CBOWDataset(Dataset):
    def __init__(self, corpus, window_size=2):
        self.corpus = corpus
        self.window_size = window_size
        self.data = self.generate_data()

    def generate_data(self):
        data = []
        for sentence in self.corpus:
            words = sentence.split()
            for i, target_word in enumerate(words):
                context = [words[j] for j in range(max(0, i - self.window_size), min(len(words), i + self.window_size + 1)) if j != i]
                data.append((context, target_word))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, target_word = self.data[idx]
        context_idx = [vocab[word] for word in context]
        target_idx = vocab[target_word]
        return torch.tensor(context_idx), torch.tensor(target_idx)

# CBOW Model
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x).mean(dim=1)  # Average embedding of context words
        x = self.linear(x)
        return x

In [ ]:
# Training parameters
vocab_size = len(vocab)
embedding_dim = 5
window_size = 2
batch_size = 4
lr = 0.01
num_epochs = 10

# Create dataset and dataloader
dataset = CBOWDataset(corpus, window_size=window_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function, and optimizer
model = CBOW(vocab_size, embedding_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [ ]:
model

In [ ]:
model.embedding.weight

In [ ]:
# Get word embeddings
word_embeddings = model.embedding.weight.detach().numpy()

# Print word embeddings
for idx, word in idx_to_word.items():
    print(f"{word}: {word_embeddings[idx]}")


## Regression with RNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Example data: a sequence of numbers
data = torch.arange(1, 11, dtype=torch.float32).view(-1, 1)
sequence_length = 3  # Length of each sequence
data

In [ ]:
def create_inout_sequences(input_data, seq_length):
    inout_seq = []
    L = len(input_data)
    for i in range(L-seq_length):
        seq = input_data[i:i+seq_length]
        label = input_data[i+seq_length:i+seq_length+1]
        inout_seq.append((seq, label))
    return inout_seq

seq_data = create_inout_sequences(data, sequence_length)
for seq, labels in seq_data:
  print(seq, labels)
  break

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=10, output_size=1):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)  # Initial hidden state

        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Fully connected layer on the last time step
        # according to chatgpt
        # this equals
        # out = self.fc(hn.squeeze(0))  # Using hn instead of out[:, -1, :]
        # i dont think its true for bidirectional coz output has last layer only (all t).
        # h as last time step (both layers)

        # a single number
        return out

model = SimpleRNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the Model
epochs = 100
for epoch in range(epochs):
    for seq, labels in seq_data:
        optimizer.zero_grad()
        y_pred = model(seq.unsqueeze(0))  # Add batch dimension
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

## Language Generation with RNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import random

# Define constants
START_TOKEN = "<START>"
END_TOKEN = "<END>"
PAD_TOKEN = "<PAD>"
BATCH_SIZE = 2
NUM_EPOCHS = 10
EMBEDDING_DIM = 100
HIDDEN_DIM = 128

# Define dataset class
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Function to build vocabulary and tokenizer
def build_vocab_and_tokenizer(data):
    words = [word for sentence in data for word in sentence.split()]
    word_counts = Counter(words)
    vocab = [word for word, count in word_counts.items()]
    vocab.append(START_TOKEN)
    vocab.append(END_TOKEN)
    vocab.append(PAD_TOKEN)
    tokenizer = {word: idx for idx, word in enumerate(vocab)}
    return vocab, tokenizer

# Function to convert text to tensor indices
def text_to_tensor(text, tokenizer):
    tensor = [tokenizer.get(word, tokenizer[PAD_TOKEN]) for word in text.split()]
    return tensor

# Custom collate function to handle variable length sequences
def collate_fn(batch):
    texts = [item for item in batch]
    tensor_texts = [torch.LongTensor(text_to_tensor(text, tokenizer)) for text in texts]
    padded_texts = pad_sequence(tensor_texts, batch_first=True, padding_value=tokenizer[PAD_TOKEN])
    return padded_texts

# Define the RNN model
class RNNLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(RNNLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        print('Batch size: ', x.size())
        embedded = self.embedding(x)
        print('Embeddings size: ', embedded.size())

        output, _ = self.rnn(embedded)
        print('Output: ', output.size())
        output = self.fc(output)
        print('Final size: ', output.size())

        # i think outputting all because we are modelling language at each step
        return output

# Prepare data
data = docs.copy() # from above

# Split data into train and test
random.shuffle(data)
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

# Build vocabulary and tokenizer
vocab, tokenizer = build_vocab_and_tokenizer(train_data)

# Define datasets and dataloaders
train_dataset = TextDataset(train_data)
test_dataset = TextDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Initialize model, loss function, and optimizer
model = RNNLanguageModel(len(vocab), EMBEDDING_DIM, HIDDEN_DIM)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0.0
    for batch in train_loader:
        optimizer.zero_grad()
        output = model(batch[:, :-1])  # Ignore the last token
        target = batch[:, 1:]  # Predict next token

        # all tokens of entire batch contribute to loss
        loss = criterion(output.reshape(-1, len(vocab)), target.reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader)}")

# Evaluation loop
model.eval()
total_loss = 0.0
with torch.no_grad():
    for batch in test_loader:
        output = model(batch[:, :-1])  # Ignore the last token
        target = batch[:, 1:]  # Predict next token
        loss = criterion(output.reshape(-1, len(vocab)), target.reshape(-1))
        total_loss += loss.item()
print(f"Test Loss: {total_loss/len(test_loader)}")

# Inference function
def generate_text(model, tokenizer, max_length=20):
    model.eval()
    current_token = tokenizer[START_TOKEN]
    generated_text = [current_token]
    while len(generated_text) < max_length:
        input_tensor = torch.LongTensor([[current_token]])
        output = model(input_tensor)
        probabilities = torch.softmax(output[0, -1], dim=0)
        next_token = torch.multinomial(probabilities, num_samples=1).item()
        if next_token == tokenizer[END_TOKEN]:
            break
        generated_text.append(next_token)
        current_token = next_token
    return ' '.join([vocab[token] for token in generated_text])

# Generate text
generated_text = generate_text(model, tokenizer, max_length=20)
print("Generated Text:", generated_text)


## GRU for Text Generation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

START_TOKEN = "<START>"
END_TOKEN = "<END>"
PAD_TOKEN = "<PAD>"


# Define the dataset
class LanguageDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# Function to tokenize the text and build vocabulary
def build_vocab(texts):
    vocab = set()
    for text in texts:
        vocab.update(text.split())
    vocab = list(vocab)
    vocab.append(START_TOKEN)
    vocab.append(END_TOKEN)
    vocab.append(PAD_TOKEN)
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    idx_to_word = {idx: word for idx, word in enumerate(vocab)}
    return vocab, word_to_idx, idx_to_word

# Function to encode text into tensor sequence
def encode_text(text, word_to_idx):
    return torch.tensor([word_to_idx[word] for word in text.split()])

# Function to decode tensor sequence into text
def decode_sequence(sequence, idx_to_word):
    return ' '.join([idx_to_word[idx.item()] for idx in sequence])

# Define the GRU language model
class GRULanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(GRULanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.gru(embedded)
        output = self.fc(output)
        return output

# Function to handle variable length sequences in DataLoader
def collate_fn(batch):
    texts = [item for item in batch]
    encoded_texts = [encode_text(text, word_to_idx) for text in texts]
    padded_texts = pad_sequence(encoded_texts, batch_first=True, padding_value=PAD_IDX)
    return padded_texts

# Define training and evaluation loop
def train(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            inputs = batch[:, :-1]
            targets = batch[:, 1:]
            outputs = model(inputs)
            loss = criterion(outputs.permute(0, 2, 1), targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

# Define generator function for inference
def generate_text(model, start_token_idx, end_token_idx, max_length):
    model.eval()
    with torch.no_grad():
        current_token = torch.tensor([start_token_idx]).unsqueeze(0)
        generated_text = []
        while len(generated_text) < max_length:
            output_probs = model(current_token)

            # use logits of the last elemet in the sequence
            _, next_token = torch.max(output_probs[:, -1, :], dim=1)
            if next_token.item() == end_token_idx:
                break
            generated_text.append(next_token.item())
            current_token = torch.cat((current_token, next_token.unsqueeze(0)), dim=1)
        return generated_text

# Example usage
texts = docs.copy()#["This is an example text.", "Another example text."]
vocab, word_to_idx, idx_to_word = build_vocab(texts)
PAD_IDX = len(vocab)
vocab_size = len(vocab) + 1
embedding_dim = 100
hidden_dim = 128
start_token_idx = vocab.index('<START>')
end_token_idx = vocab.index('<END>')
max_length = 5

dataset = LanguageDataset(texts)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

model = GRULanguageModel(vocab_size, embedding_dim, hidden_dim)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model, train_loader, criterion, optimizer, num_epochs=10)

generated_text = generate_text(model, start_token_idx, end_token_idx, max_length)
print(decode_sequence(torch.tensor(generated_text), idx_to_word))


## Another GRU for Text Generation (duplicated)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np

# Define your dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # no labels coz self-supervised
        return self.data[idx]

# Define your model architecture
class MyModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(MyModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        print('Embedding size: ', embedded.size())
        output, hidden = self.gru(embedded, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        # 1 = n_layers
        return torch.zeros(1, batch_size, self.hidden_size).to(device)

# Define your data processing functions
def build_vocab(data):
    word_counter = Counter()
    for sentence in data:
        word_counter.update(sentence.split())
    vocab = ['<pad>', '<start>', '<end>', '<unk>'] + [word for word, _ in word_counter.most_common()]
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for word, idx in word2idx.items()}
    return vocab, word2idx, idx2word

def encode_sentence(sentence, word2idx):
    return [word2idx.get(word, word2idx['<unk>']) for word in sentence.split()]

def collate_fn(batch):
    sentences = [torch.LongTensor(item) for item in batch]
    padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=0)
    return padded_sentences

# Define your training loop
def train(model, train_loader, criterion, optimizer, batch_size, device):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs = batch[:, :-1].to(device)
        targets = batch[:, 1:].to(device)
        hidden = model.init_hidden(batch_size)
        optimizer.zero_grad()
        outputs, _ = model(inputs, hidden)
        loss = criterion(outputs.view(-1, outputs.size(2)), targets.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Define your evaluation loop
def evaluate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch[:, :-1].to(device)
            targets = batch[:, 1:].to(device)
            hidden = model.init_hidden(len(batch))
            outputs, _ = model(inputs, hidden)
            loss = criterion(outputs.view(-1, outputs.size(2)), targets.contiguous().view(-1))
            total_loss += loss.item()
    return total_loss / len(val_loader)

# Define your inference function
def generate_sentence(model, start_token_idx, end_token_idx, max_length, batch_size, device):
    model.eval()
    with torch.no_grad():
        current_token_idx = start_token_idx
        sentence = [current_token_idx]
        hidden = model.init_hidden(1)
        for _ in range(max_length):
            inputs = torch.LongTensor([[current_token_idx]]).to(device)
            outputs, hidden = model(inputs, hidden)
            predicted_token_idx = outputs.argmax().item()
            if predicted_token_idx == end_token_idx:
                break
            sentence.append(predicted_token_idx)
            current_token_idx = predicted_token_idx
    return sentence

In [ ]:
# Example usage
#data = ['<start> hello how are you <end>', '<start> i am fine thank you <end>']
data = docs.copy()
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

vocab, word2idx, idx2word = build_vocab(train_data)
train_data_encoded = [encode_sentence(sentence, word2idx) for sentence in train_data]
val_data_encoded = [encode_sentence(sentence, word2idx) for sentence in val_data]

train_dataset = MyDataset(train_data_encoded)
val_dataset = MyDataset(val_data_encoded)

batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyModel(vocab_size=len(vocab), embed_size=50, hidden_size=100)
model.to(device)

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, batch_size, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Example inference
start_token_idx = word2idx['<start>']
end_token_idx = word2idx['<end>']
generated_sentence = generate_sentence(model, start_token_idx, end_token_idx, 10, batch_size, device)
generated_sentence = ' '.join([idx2word[idx] for idx in generated_sentence])
print(generated_sentence)

## LSTM for Sentiment Analysis

In [ ]:
import pandas as pd

# IMDb reviews data as a dictionary
imdb_reviews = {
    "review": [
        "The Shawshank Redemption is an incredible movie. The acting is superb and the storyline is captivating.",
        "I found the plot to be confusing and the acting to be mediocre. Overall, I was disappointed with the film.",
        "Great movie! The special effects were amazing and the characters were well-developed.",
        "The dialogue felt forced and unrealistic. I couldn't connect with any of the characters.",
        "The Godfather is a masterpiece. The performances are outstanding and the direction is flawless.",
        "The pacing of the film was too slow, and I found myself losing interest halfway through.",
        "Titanic is a classic love story that will tug at your heartstrings. I highly recommend it.",
        "The acting was wooden and the plot was predictable. I was not impressed.",
        "Jurassic Park is a thrilling adventure that keeps you on the edge of your seat from start to finish.",
        "I felt the movie lacked depth and the characters were one-dimensional. Overall, it was underwhelming."
    ],
    "sentiment": [
        "positive",
        "negative",
        "positive",
        "negative",
        "positive",
        "negative",
        "positive",
        "negative",
        "positive",
        "negative"
    ]
}

df = pd.DataFrame(imdb_reviews)
df = pd.concat([df, df]).reset_index(drop=True)
df

In [ ]:
X, y = df['review'], df['sentiment']

from sklearn.model_selection import train_test_split

# can also use random_split() by torch on top of pytorch's dataset
# like: train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
y_train.value_counts()

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
import string

def preprocess_word(w):
    return w.lower()

def build_vocab(X_train):
    stop_words = set(stopwords.words('english'))
    word_list = set()

    # Building the vocabulary
    for sent in X_train:
        for word in sent.lower().split():
            word = preprocess_word(word)
            if word not in stop_words and word != '' and word not in string.punctuation:
                word_list.add(word)

    vocab = {w: i for i, w in enumerate(word_list)}
    return vocab

def tokenize(sentences):
    tokenized_list = []
    for sent in sentences:
        tokenized_words = [vocab[preprocess_word(w)] for w in sent.lower().split() if preprocess_word(w) in vocab]
        if len(tokenized_words) == 0:
          print(sent)
          continue
        tokenized_list.append(torch.tensor(tokenized_words))
    return tokenized_list

def encode_labels(labels):
  return torch.tensor([1 if label == 'positive' else 0 for label in labels])


def preprocess(X, y):
    tokenized_text = tokenize(X.values)
    encoded_labels = encode_labels(y.values)
    return tokenized_text, encoded_labels

vocab = build_vocab(X_train)

# Tokenize/encode
X_train_tokenized, y_train_tokenized = preprocess(X_train, y_train)
X_test_tokenized, y_test_tokenized = preprocess(X_test, y_test)

print(X_train_tokenized)
print(y_train_tokenized)
print(X_test_tokenized)
print(y_test_tokenized)

In [ ]:
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset

# create Tensor datasets

# using custom class for variable input
train_data = SequenceDataset(X_train_tokenized, y_train_tokenized)
valid_data = SequenceDataset(X_test_tokenized, y_test_tokenized)

# dataloaders
batch_size = 2

def collate_fn(batch):
    sentences, labels = zip(*batch)
    lengths = [len(sent) for sent in sentences]
    padded = pad_sequence(sentences, batch_first=True, padding_value=0)
    return padded, torch.tensor(lengths), torch.tensor(labels)

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, collate_fn=collate_fn)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, collate_fn=collate_fn)

# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_lengths, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample output: \n', sample_y)
print('Sample sample_lengths: \n', sample_lengths)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
class SentimentRNN(torch.nn.Module):
    def __init__(self, no_layers, vocab_size,hidden_dim,embedding_dim,output_dim,drop_prob=0.5):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.no_layers = no_layers
        self.vocab_size = vocab_size

        # embedding and LSTM layers
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)

        #lstm: DOES NOT DEPEND ON SEQUENCE LENGTH. SHARED WEIGHTS.
        self.lstm = torch.nn.LSTM(
            input_size=embedding_dim,
            hidden_size=self.hidden_dim,
            num_layers=no_layers,
            batch_first=True #bidirectional=True
        )

        # dropout layer
        self.dropout = torch.nn.Dropout(0.3)

        # linear and sigmoid layer
        self.fc = torch.nn.Linear(self.hidden_dim, output_dim) # *2 because of bidirectional
        self.sig = torch.nn.Sigmoid()

    def forward(self, x, lengths, hidden):
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)  # shape: B x S x Feature   since batch = True

        # using packed sequences
        packed_embed = pack_padded_sequence(embeds, lengths, batch_first=True, enforce_sorted=False)

        # handles feedback loop and processes all the tokens in a sequence
        lstm_out, hidden = self.lstm(packed_embed, hidden)

        # for BILSTM: concatenate the final forward and backward hidden states
        # forward even indices, backward odd`
        # hidden_concat = torch.cat((hidden[0][-2, :, :], hidden[0][-1, :, :]), dim=1)

        # Unpack the PackedSequence
        lstm_out, _ = torch.nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        """
        Reshapes the lstm_out tensor to have shape (batch_size * sequence_length, hidden_dim).
        The -1 is a placeholder that infers the appropriate size automatically based on the
        other dimension (hidden_dim). This essentially flattens the tensor so that each time step
        for each batch item is treated as an independent sample.
        """

        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)

        sig_out = sig_out[:, -1] # get element of each batch I guess

        # return last sigmoid output and hidden state
        return sig_out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM

        # twice layers for BILSTM
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

    # optionally:
    # def init_weights(self):
    #     for param in self.parameters():
    #         nn.init.kaiming_uniform_(param, a=math.sqrt(5))

In [ ]:
no_layers = 2
vocab_size = len(vocab) + 1 # extra 1 for padding????
embedding_dim = 64
output_dim = 1
hidden_dim = 12

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = SentimentRNN(no_layers, vocab_size, hidden_dim, embedding_dim, output_dim, drop_prob=0.5)
model.to(device)

In [ ]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()}\n")

In [ ]:
# function to calculate correct labels
def acc(pred,label):
    # simple way to convert probs into classes
    pred = torch.round(pred.squeeze())

    # count correct predictions
    return torch.sum(pred == label.squeeze()).item()

In [ ]:
clip = 5 # gradient clipping for exploding gradients
epochs = 5
valid_loss_min = np.Inf
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]


for epoch in range(epochs):
    train_losses = []
    train_acc = 0.0

    # layers like dropout are enabled. gradients ar eclcualed. batch normalziation uses batch statistics.
    model.train()

    for inputs, lengths, labels in train_loader:
        # move to GPU
        inputs, lengths, labels = inputs.to(device), lengths.to(device), labels.to(device)

        # Creating new variables for the hidden state, otherwise we'd backprop through the entire training history
        # h = tuple([each.data for each in h])

        # why not just init wihtin the loop
        # initialize hidden states and cell states of LSTM
        h = model.init_hidden(batch_size)

        # clear grad to prevent gradient accumulation across batches.
        model.zero_grad()

        # forward() pass. maintain operations gradient functions in DAG .grad_fn
        output, _ = model(inputs, lengths, h)  # h not used outside

        # calculate the loss
        loss = criterion(output, labels.float())
        train_losses.append(loss.item())

        # calculate gradients through backpropagation into .grad, applies chain rule
        loss.backward()

        #`clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # update weights
        optimizer.step()

        # calculating accuracy (correct labels)
        accuracy = acc(output, labels)
        train_acc += accuracy # actually correct predictions


    val_losses = []
    val_acc = 0.0

    # disable some operations
    model.eval()

    # no gradient calculations needed
    with torch.no_grad():
      for inputs, lengths, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        #val_h = tuple([each.data for each in val_h])
        val_h = model.init_hidden(batch_size)

        output, val_h = model(inputs, lengths, val_h)

        val_loss = criterion(output, labels.float())

        val_losses.append(val_loss.item())
        accuracy = acc(output,labels)
        val_acc += accuracy

    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)
    epoch_train_acc = train_acc/len(train_loader.dataset)
    epoch_val_acc = val_acc/len(valid_loader.dataset)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch+1}')
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')

    if epoch_val_loss <= valid_loss_min:
        torch.save(model.state_dict(), 'state_dict.pt')
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25*'==')


In [ ]:
def predict_text(text):
  model.eval()

  word_seq = np.array([vocab[preprocess_word(word)] for word in text.split() if preprocess_word(word) in vocab])
  length = torch.tensor([len(word_seq)])
  word_seq = np.expand_dims(word_seq, axis=0)
  inputs = torch.from_numpy(word_seq).to(device)
  batch_size = 1
  h = model.init_hidden(batch_size)
  # h = tuple([each.data for each in h]) # maybe unnecessary
  with torch.no_grad():
    output, h = model(inputs, length, h)
  probability = output.item()
  return probability

index = 0
predict_text('this movie sucks.')

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

## CNN for Sentiment Analysis

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
from collections import Counter
import itertools
from sklearn.model_selection import train_test_split

# Sample data
texts = [
    "I love this movie",
    "This film was terrible",
    "Absolutely fantastic",
    "Worst movie ever",
    "I enjoyed it",
    "Not my favorite",
    "Best movie I have seen",
    "Could be better",
    "I would watch it again",
    "Awful experience"
]

labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1 for positive, 0 for negative

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenization
tokenized_train_texts = [text.split() for text in train_texts]

# Vocabulary creation
train_vocab = Counter(itertools.chain(*tokenized_train_texts))
train_vocab = {word: i + 2 for i, (word, _) in enumerate(train_vocab.most_common())}  # Start indices from 2

# I don't think this info is needed for tasks other than langauge modelling
train_vocab['<PAD>'] = 0
train_vocab['<UNK>'] = 1

# Convert texts to sequences of indices
def text_to_sequence(text, vocab):
    return [vocab.get(word, vocab['<UNK>']) for word in text]

# Padding
def pad_sequence(seq, max_len):
    return seq + [train_vocab['<PAD>']] * (max_len - len(seq))

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, sequences, labels):
        self.data = sequences
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def collate_fn(batch):
    print('Batch: ')
    print(batch)
    print('Unpacked:')
    print(*batch)
    print('Zipped:')
    print(zip(*batch))
    texts, labels = zip(*batch)
    sequences = [pad_sequence(text, max_len) for text in texts]
    return torch.tensor(sequences, dtype=torch.long), torch.tensor(labels, dtype=torch.long)


train_sequences = [text_to_sequence(text, train_vocab) for text in tokenized_train_texts]
max_len = max(len(seq) for seq in train_sequences)
train_dataset = SentimentDataset(train_sequences, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
for batch in train_dataloader:
  tokens, labels = batch
  print(tokens, labels)
  break

In [ ]:
class SimpleCNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(SimpleCNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # treat embedding dimension as in_channels, kernel size is just a window for context words
        self.conv = nn.Conv1d(in_channels=embed_dim, out_channels=50, kernel_size=3)  # 50 filters, kernel size 3
        self.fc = nn.Linear(50 * (max_len - 3 + 1), num_classes)  # Fully connected layer

    def forward(self, x):
        # x: (batch_size, max_len)
        x = self.embedding(x)  # (batch_size, max_len, embed_dim)
        x = x.permute(0, 2, 1)  # Rearrange to (batch_size, embed_dim, max_len)
        x = F.relu(self.conv(x))  # Apply convolution: (batch_size, 50, max_len-3+1)
        x = x.view(x.size(0), -1)  # Flatten: (batch_size, 50 * (max_len-3+1))
        x = self.fc(x)  # Fully connected layer: (batch_size, num_classes)
        return x

vocab_size = len(train_vocab)
embed_dim = 50  # Reduced embedding dimension for simplicity
num_classes = 2

model = SimpleCNNModel(vocab_size, embed_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    print('==='*50)
    for batch_data, batch_labels in train_dataloader:
        outputs = model(batch_data)  # logits
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Sample inference
sample_text = "I really love this movie"
sample_sequence = text_to_sequence(sample_text.split(), train_vocab)
sample_sequence_padded = pad_sequence(sample_sequence, max_len)
sample_data = torch.tensor([sample_sequence_padded], dtype=torch.long)
output = model(sample_data)  # logits

print(output)

_, predicted = torch.max(output, 1)
print(f'Predicted: {predicted.item()}')

In [ ]:
#

# Pipeline / Sklearn

In [ ]:
import pandas as pd
X = pd.DataFrame(
    {'city': ['London', 'London', 'Paris', 'Sallisaw'],
     'state' : ['NY', 'NJ', 'ON', 'QB'],
     'title': ["His Last Bow", "How Watson Learned the Trick",
               "A Moveable Feast", "The Grapes of Wrath"],
     'expert_rating': [5, 3, 4, 5],
     'user_rating': [4, 5, 4, 3]})

X

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector

# using columntrasnformer avoids data leakage that happens if we preprocess before splitting/sklearn training.
# Also, can be parameterized.

column_trans = ColumnTransformer(
    [('categories', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['city']), # list for most transformers like this one
     ('title_bow', CountVectorizer(), 'title'),
     ('standard_scaler', StandardScaler(), make_column_selector(dtype_include=np.number))
    ],
    remainder='drop'
)

# NOTE:
# shouldn't fit before split
column_trans.fit(X)

column_trans.get_feature_names_out()

feature_set = column_trans.transform(X)#.toarray()
feature_set

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import set_config
import joblib

# Define the pipeline
pipeline = make_pipeline(column_trans, RandomForestClassifier())

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    X.drop(columns=['user_rating']),
    X['user_rating'],
    test_size=0.5,
    random_state=42
)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Save the pipeline for future use
joblib.dump(pipeline, 'pipeline.pkl')

# Later, load the pipeline and use it to make predictions on new data
pipeline_loaded = joblib.load('pipeline.pkl')

# Make predictions on new data
predictions = pipeline_loaded.predict(X_valid)
predictions

In [ ]:
from sklearn.model_selection import cross_val_score

# can use cross_val_score for pileine

scores = cross_val_score(pipeline, X.drop(columns=['user_rating']), X['user_rating'], cv=2)
print(f"Mean accuracy: {scores.mean():.2f}+/-{scores.std():.2f}")

# LDA

In [ ]:
from hdbscan import HDBSCAN
embedding_model = "all-MiniLM-L12-v2"

In [ ]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5,
                                min_df = 1)

dtm_tf = tf_vectorizer.fit_transform(docs)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs)

In [ ]:
dense_matrix = dtm_tf.toarray()
dense_matrix

In [ ]:
dense_matrix_tfidf = dtm_tfidf.toarray()
print(dense_matrix_tfidf.shape)
dense_matrix_tfidf

In [ ]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=4, random_state=0)
lda_tf.fit(dtm_tf)

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=4, random_state=0)
lda_tfidf.fit(dtm_tfidf)

In [ ]:
from pyLDAvis import lda_model as psk

In [ ]:
scipy.__version__

In [ ]:
from gensim import corpora, models

# # Example corpus of documents
# corpus = [
#     "This is the first document",
#     "This document is the second document",
#     "And this is the third one",
#     "Is this the first document?"
# ]

corpus = docs.copy()

# Preprocessing the corpus
tokenized_corpus = [document.lower().split() for document in corpus]

# Creating the dictionary
dictionary = corpora.Dictionary(tokenized_corpus)

In [ ]:
tokenized_corpus

In [ ]:
dictionary

In [ ]:
print(dictionary.token2id)

In [ ]:
# Creating the document-term-frequency matrix
doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in tokenized_corpus]
doc_term_matrix

In [ ]:
# Training the LDA model
lda_model = models.LdaModel(
    corpus=doc_term_matrix,
    id2word=dictionary,
    num_topics=3,
    passes=10
)

# Print the topics and their associated keywords
for topic_id, topic_keywords in lda_model.print_topics():
    print(f"Topic ID: {topic_id}")
    print(f"Keywords: {topic_keywords}")
    print()


In [ ]:
0.039 + 0.025 + 0.021 + 0.019 + 0.018 + 0.017 + 0.017 + 0.017 + 0.017 + 0.017 + 0.016 + 0.016

In [ ]:
# probabilityies of classes
count = 0
for doc in lda_model[doc_term_matrix]:
  print('doc: ', count, doc)
  count += 1

In [ ]:
import pyLDAvis.gensim

# # Example corpus of documents
# corpus = [
#     "This is the first document",
#     "This document is the second document",
#     "And this is the third one",
#     "Is this the first document?"
# ]

# # Preprocessing the corpus
# tokenized_corpus = [document.lower().split() for document in corpus]

# # Creating the dictionary
# dictionary = corpora.Dictionary(tokenized_corpus)

# # Creating the document-term matrix
# doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in tokenized_corpus]

# # Training the LDA model
# lda_model = models.LdaModel(
#     corpus=doc_term_matrix,
#     id2word=dictionary,
#     num_topics=3,
#     passes=10
# )

# Prepare the data for visualization
lda_vis_data = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)

# Display the interactive topic visualization
pyLDAvis.display(lda_vis_data)


In [ ]:
import pandas as pd
from bertopic import BERTopic
import pyLDAvis

# # Example corpus of documents
# corpus = [
#     "This is the first document",
#     "This document is the second document",
#     "And this is the third one",
#     "Is this the first document?"
# ]

# Initialize and train the Bertopic model
bertopic_model = BERTopic()
topics, _ = bertopic_model.fit_transform(docs)

# Convert the topics to a Pandas DataFrame
df_topics = pd.DataFrame(topics, columns=["Topic"])

# Generate the topic visualization using pyLDAvis
lda_vis_data = pyLDAvis.prepare(
    df_topics,
    bertopic_model.transform(docs),
    bertopic_model.get_topics(),
    df_topics["Topic"].value_counts(),
    R=10,  # Number of relevant terms to display per topic
    lambda_step=0.01
)

# Display the interactive topic visualization
pyLDAvis.display(lda_vis_data)


In [ ]:
%pip show pyLDAvis